# 9

In [87]:
text = "YELLOW SUBMARINE"

In [17]:
class PKCS7:
    def __init__(self, block_size):
        self.block_size = block_size

    def pad(self, text):
        n = self.block_size - len(text) % self.block_size
        return text + bytes([n] * n)
    
    def unpad(self, text):
        n = text[-1]
        return text[:-n]

In [90]:
pkcs7 = PKCS7(20)
pkcs7.pad(text.encode())

b'YELLOW SUBMARINE\x04\x04\x04\x04'

# 10

In [1]:
import base64
key = "YELLOW SUBMARINE"
with open('./cryptopals.com_static_challenge-data_10.txt', 'r') as f:
    file = base64.b64decode(f.read().replace('\n', '').encode())

In [2]:
from utils import xor, PKCS7, AES_CBC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

blockSize = int(algorithms.AES(key.encode()).block_size/8)

In [3]:
aes_cbc = AES_CBC(key = key.encode(),
                  IV = b'\x00'*blockSize)
print(aes_cbc.decrypt(file).decode())

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


# 11

In [7]:
import os
from utils import AES_CBC, PKCS7
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

def encryption_oracle(plainText):
    lengthRandomBytes = 5 + int(int.from_bytes(os.urandom(1), byteorder='big')* (6/256))
    plainText = os.urandom(lengthRandomBytes) + plainText + os.urandom(lengthRandomBytes)

    blockSize = 16
    key = os.urandom(blockSize)

    if int.from_bytes(os.urandom(1), byteorder='big')/256 < 0.5:
        cipher = AES_CBC(key, IV=os.urandom(blockSize))
        return cipher.encrypt(plainText)
    else:
        cipher = Cipher(algorithm=algorithms.AES(key),
                        mode=modes.ECB())
        padding = PKCS7(blockSize)
        plainText = padding.pad(plainText)
        encryptor = cipher.encryptor()
        return encryptor.update(plainText)

In [8]:
import base64
key = "YELLOW SUBMARINE"
with open('./cryptopals.com_static_challenge-data_10.txt', 'r') as f:
    file = base64.b64decode(f.read().replace('\n', '').encode())

In [9]:
from utils import check_repetitions_ECB
from collections import Counter

experiments = []
for i in range(0, 1000):
    if check_repetitions_ECB(encryption_oracle(b'0'*256), 16):
        experiments.append('ECB mode')
    else:
        experiments.append('CBC mode')

counts = Counter(experiments)
results = {key: value/len(experiments) for key, value in counts.items()}
print(results)

{'CBC mode': 0.514, 'ECB mode': 0.486}


# 12

In [26]:
import os
import base64
from utils import PKCS7
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

blockSize = 16
KEY = os.urandom(blockSize)

def oracle(plainText):
    header = base64.b64decode("Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg"
                           "aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq"
                           "dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg"
                           "YnkK")
    plainText += header
    cipher = Cipher(algorithm=algorithms.AES(KEY),
                    mode=modes.ECB())
    padding = PKCS7(blockSize)
    plainText = padding.pad(plainText)
    encryptor = cipher.encryptor()
    return encryptor.update(plainText)

In [31]:
stop = False
startingLen = len(oracle(b''))
i = 1
while not stop:
    newLen = len(oracle(b'A'*i))
    if newLen != startingLen:
        stop = True
    else:
        i += 1
blockSizeFound = newLen - startingLen
print(f'Padding length = {blockSizeFound}')

Padding length = 16


In [33]:
from utils import check_repetitions_ECB
print(f"ECB mode? {check_repetitions_ECB(oracle(b'A'*blockSize*2), blockSizeFound)}")

ECB mode? True
